# Trying to figure out how to compare models, not super interesting.  
## See: Content Models.ipynb, Explicit Similarity Models.ipynb, Implicit Similarity Models.ipynb

In [1]:
# if you don't already have it
# !pip install turicreate
import turicreate as tc
from turicreate.toolkits.recommender.util import precision_recall_by_user

In [2]:
# docs: https://apple.github.io/turicreate/docs/userguide/recommender/

In [3]:
# data
actions = tc.SFrame.read_csv('../data/raw_data/ratings.csv')
books = tc.SFrame.read_csv('../data/raw_data/books.csv')
to_read = tc.SFrame.read_csv('../data/raw_data/to_read.csv')

Finished parsing file /home/jovyan/work/MIDS207-Final-Project/data/raw_data/ratings.csv

Parsing completed. Parsed 100 lines in 6.1315 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 4373238 lines. Lines per second: 1.78034e+06

Finished parsing file /home/jovyan/work/MIDS207-Final-Project/data/raw_data/ratings.csv

...

Finished parsing file /home/jovyan/work/MIDS207-Final-Project/data/raw_data/to_read.csv

Parsing completed. Parsed 100 lines in 0.17579 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /home/jovyan/work/MIDS207-Final-Project/data/raw_data/to_read.csv

Parsing completed. Parsed 912705 lines in 0.207298 secs.

In [4]:
# train and test data for explicit models
high_rated_data = actions[actions["rating"] >= 4]
low_rated_data = actions[actions["rating"] < 4]
train_data_1, test_data = tc.recommender.util.random_split_by_user(
                                    high_rated_data, user_id='user_id', item_id='book_id')
train_data = train_data_1.append(low_rated_data)

In [5]:
# train and test data for implicit models
to_read_train, to_read_test = tc.recommender.util.random_split_by_user(to_read, user_id='user_id', item_id='book_id')

# Popularity (Baseline)

In [7]:
popularity_model = tc.recommender.popularity_recommender.create(train_data, user_id='user_id', item_id='book_id', target='rating')
popularity_model.evaluate_precision_recall(test_data, exclude_known=True, verbose=False)

Preparing data set.

Data has 5960837 observations with 53424 users and 10000 items.

Data prepared in: 3.05367s

5960837 observations to process; with 10000 unique items.

{'precision_recall_by_user': Columns:
 	user_id	int
 	cutoff	int
 	precision	float
 	recall	float
 	count	int
 
 Rows: 18000
 
 Data:
 +---------+--------+-----------+--------+-------+
 | user_id | cutoff | precision | recall | count |
 +---------+--------+-----------+--------+-------+
 |    2    |   1    |    0.0    |  0.0   |   8   |
 |    2    |   2    |    0.0    |  0.0   |   8   |
 |    2    |   3    |    0.0    |  0.0   |   8   |
 |    2    |   4    |    0.0    |  0.0   |   8   |
 |    2    |   5    |    0.0    |  0.0   |   8   |
 |    2    |   6    |    0.0    |  0.0   |   8   |
 |    2    |   7    |    0.0    |  0.0   |   8   |
 |    2    |   8    |    0.0    |  0.0   |   8   |
 |    2    |   9    |    0.0    |  0.0   |   8   |
 |    2    |   10   |    0.0    |  0.0   |   8   |
 +---------+--------+-----------+--------+-------+
 [18000 rows x 5 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and colum

# Explicit Classifiers

## Similarity

In [6]:
item_similarity_model = tc.recommender.item_similarity_recommender.create(train_data, user_id='user_id', item_id='book_id', target='rating')
item_similarity_model.evaluate_precision_recall(test_data, exclude_known=True, verbose=False)

Preparing data set.

Data has 5960837 observations with 53424 users and 10000 items.

Data prepared in: 2.87628s

Training model from provided data.

Gathering per-item and per-user statistics.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 5.53896s

{'precision_recall_by_user': Columns:
 	user_id	int
 	cutoff	int
 	precision	float
 	recall	float
 	count	int
 
 Rows: 18000
 
 Data:
 +---------+--------+---------------------+--------+-------+
 | user_id | cutoff |      precision      | recall | count |
 +---------+--------+---------------------+--------+-------+
 |    2    |   1    |         0.0         |  0.0   |   8   |
 |    2    |   2    |         0.5         | 0.125  |   8   |
 |    2    |   3    |  0.3333333333333333 | 0.125  |   8   |
 |    2    |   4    |         0.25        | 0.125  |   8   |
 |    2    |   5    |         0.2         | 0.125  |   8   |
 |    2    |   6    | 0.16666666666666666 | 0.125  |   8   |
 |    2    |   7    | 0.14285714285714285 | 0.125  |   8   |
 |    2    |   8    |        0.125        | 0.125  |   8   |
 |    2    |   9    |  0.1111111111111111 | 0.125  |   8   |
 |    2    |   10   |         0.1         | 0.125  |   8   |
 +---------+--------+---------------------+--------+-------+
 [18000 rows

In [8]:
item_similarity_model.recommend(['1']).join(books)

user_id,book_id,score,rank,goodreads_book_id,best_book_id,work_id,books_count,isbn
1,5,0.08824248242582011,1,4671,4671,245494,1356,743273567
1,8,0.08291644622118045,2,5107,5107,3036731,360,316769177
1,14,0.06432155768076579,6,7613,7613,2207778,896,452284244
1,15,0.07325095753384452,3,48855,48855,3532896,710,553296981
1,26,0.0635126773108784,7,968,968,2982101,350,307277674
1,28,0.07093797127405803,4,7624,7624,2766512,458,140283331
1,63,0.06008427927636693,9,6185,6185,1565818,2498,393978893
1,64,0.05328749999021872,10,10917,10917,1639903,125,743454537
1,118,0.06941591266892914,5,7763,7763,1955658,126,143038095
1,195,0.06038214966782138,8,2728527,2728527,2754161,116,385340990


## Factorization

In [28]:
factorization_model = tc.recommender.factorization_recommender.create(train_data, user_id='user_id', item_id='book_id', target='rating')
factorization_model.evaluate_precision_recall(test_data, exclude_known=True, verbose=False)

Preparing data set.

Data has 5960837 observations with 53424 users and 10000 items.

Data prepared in: 2.3477s

Training factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 0.529099

Final training RMSE: 0.72677

{'precision_recall_by_user': Columns:
 	user_id	int
 	cutoff	int
 	precision	float
 	recall	float
 	count	int
 
 Rows: 18000
 
 Data:
 +---------+--------+-----------+--------+-------+
 | user_id | cutoff | precision | recall | count |
 +---------+--------+-----------+--------+-------+
 |    2    |   1    |    0.0    |  0.0   |   8   |
 |    2    |   2    |    0.0    |  0.0   |   8   |
 |    2    |   3    |    0.0    |  0.0   |   8   |
 |    2    |   4    |    0.0    |  0.0   |   8   |
 |    2    |   5    |    0.0    |  0.0   |   8   |
 |    2    |   6    |    0.0    |  0.0   |   8   |
 |    2    |   7    |    0.0    |  0.0   |   8   |
 |    2    |   8    |    0.0    |  0.0   |   8   |
 |    2    |   9    |    0.0    |  0.0   |   8   |
 |    2    |   10   |    0.0    |  0.0   |   8   |
 +---------+--------+-----------+--------+-------+
 [18000 rows x 5 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and colum

## Ranking Factorization

In [31]:
ranking_factorization_model = tc.recommender.ranking_factorization_recommender.create(train_data, user_id='user_id', item_id='book_id', target='rating')
ranking_factorization_model.evaluate_precision_recall(test_data, exclude_known=True, verbose=False)

Preparing data set.

Data has 5960837 observations with 53424 users and 10000 items.

Data prepared in: 2.40575s

Training ranking_factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 0.775356

Final training RMSE: 0.698427

{'precision_recall_by_user': Columns:
 	user_id	int
 	cutoff	int
 	precision	float
 	recall	float
 	count	int
 
 Rows: 18000
 
 Data:
 +---------+--------+--------------------+--------+-------+
 | user_id | cutoff |     precision      | recall | count |
 +---------+--------+--------------------+--------+-------+
 |    2    |   1    |        0.0         |  0.0   |   8   |
 |    2    |   2    |        0.0         |  0.0   |   8   |
 |    2    |   3    |        0.0         |  0.0   |   8   |
 |    2    |   4    |        0.0         |  0.0   |   8   |
 |    2    |   5    |        0.0         |  0.0   |   8   |
 |    2    |   6    |        0.0         |  0.0   |   8   |
 |    2    |   7    |        0.0         |  0.0   |   8   |
 |    2    |   8    |       0.125        | 0.125  |   8   |
 |    2    |   9    | 0.1111111111111111 | 0.125  |   8   |
 |    2    |   10   |        0.1         | 0.125  |   8   |
 +---------+--------+--------------------+--------+-------+
 [18000 rows x 5 columns]


# Implicit Classifiers

## Item Similarity

In [40]:
implicit_item_similarity_model = tc.recommender.item_similarity_recommender.create(to_read_train, user_id='user_id', item_id='book_id')
implicit_item_similarity_model.evaluate_precision_recall(to_read_test, exclude_known=True, verbose=False)

Preparing data set.

Data has 908882 observations with 48865 users and 9985 items.

Data prepared in: 0.339823s

Training model from provided data.

Gathering per-item and per-user statistics.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 3.15793s

{'precision_recall_by_user': Columns:
 	user_id	int
 	cutoff	int
 	precision	float
 	recall	float
 	count	int
 
 Rows: 14760
 
 Data:
 +---------+--------+-----------+--------+-------+
 | user_id | cutoff | precision | recall | count |
 +---------+--------+-----------+--------+-------+
 |   2224  |   1    |    0.0    |  0.0   |   1   |
 |   2224  |   2    |    0.0    |  0.0   |   1   |
 |   2224  |   3    |    0.0    |  0.0   |   1   |
 |   2224  |   4    |    0.0    |  0.0   |   1   |
 |   2224  |   5    |    0.0    |  0.0   |   1   |
 |   2224  |   6    |    0.0    |  0.0   |   1   |
 |   2224  |   7    |    0.0    |  0.0   |   1   |
 |   2224  |   8    |    0.0    |  0.0   |   1   |
 |   2224  |   9    |    0.0    |  0.0   |   1   |
 |   2224  |   10   |    0.0    |  0.0   |   1   |
 +---------+--------+-----------+--------+-------+
 [14760 rows x 5 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and colum

## Ranking Factorization

In [41]:
implicit_ranking_factorization_model = tc.recommender.ranking_factorization_recommender.create(to_read_train, user_id='user_id', item_id='book_id')
implicit_ranking_factorization_model.evaluate_precision_recall(to_read_test, exclude_known=True, verbose=False)

Preparing data set.

Data has 908882 observations with 48865 users and 9985 items.

Data prepared in: 0.34438s

Training ranking_factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training Predictive Error.

Final objective value: 1.33321

Final training Predictive Error: 0.66815

{'precision_recall_by_user': Columns:
 	user_id	int
 	cutoff	int
 	precision	float
 	recall	float
 	count	int
 
 Rows: 14760
 
 Data:
 +---------+--------+-----------+--------+-------+
 | user_id | cutoff | precision | recall | count |
 +---------+--------+-----------+--------+-------+
 |   2224  |   1    |    0.0    |  0.0   |   1   |
 |   2224  |   2    |    0.0    |  0.0   |   1   |
 |   2224  |   3    |    0.0    |  0.0   |   1   |
 |   2224  |   4    |    0.0    |  0.0   |   1   |
 |   2224  |   5    |    0.0    |  0.0   |   1   |
 |   2224  |   6    |    0.0    |  0.0   |   1   |
 |   2224  |   7    |    0.0    |  0.0   |   1   |
 |   2224  |   8    |    0.0    |  0.0   |   1   |
 |   2224  |   9    |    0.0    |  0.0   |   1   |
 |   2224  |   10   |    0.0    |  0.0   |   1   |
 +---------+--------+-----------+--------+-------+
 [14760 rows x 5 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and colum

In [37]:
tc.recommender.util.compare_models(test_data, 
                                   [popularity_model, item_similarity_model], 
                                   model_names=["Popularity", "Explicit Item Simularity"],
                                  metric='precision_recall',
                                   exclude_known_for_precision_recall=True,
                                  verbose=False)

[{'precision_recall_by_user': Columns:
  	user_id	int
  	cutoff	int
  	precision	float
  	recall	float
  	count	int
  
  Rows: 18000
  
  Data:
  +---------+--------+-----------+--------+-------+
  | user_id | cutoff | precision | recall | count |
  +---------+--------+-----------+--------+-------+
  |    2    |   1    |    0.0    |  0.0   |   8   |
  |    2    |   2    |    0.0    |  0.0   |   8   |
  |    2    |   3    |    0.0    |  0.0   |   8   |
  |    2    |   4    |    0.0    |  0.0   |   8   |
  |    2    |   5    |    0.0    |  0.0   |   8   |
  |    2    |   6    |    0.0    |  0.0   |   8   |
  |    2    |   7    |    0.0    |  0.0   |   8   |
  |    2    |   8    |    0.0    |  0.0   |   8   |
  |    2    |   9    |    0.0    |  0.0   |   8   |
  |    2    |   10   |    0.0    |  0.0   |   8   |
  +---------+--------+-----------+--------+-------+
  [18000 rows x 5 columns]
  Note: Only the head of the SFrame is printed.
  You can use print_rows(num_rows=m, num_columns=n) t

In [ ]:
# syntax for compare_models
# tc.recommender.util.compare_models(test_data, 
#                                    [factorization_model, item_similarity_model], 
#                                    model_names=["Explicit Factorization", "Explicit Item Simularity"],
#                                   metric='precision_recall',
#                                    exclude_known_for_precision_recall=True,
#                                   verbose=False)

# Content Model